In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

import utils

import nltk
from nltk.corpus import stopwords
import re

# Download stopwords if you haven't already
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edwar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
meetings_df = pd.read_csv(os.path.join(LOCAL_PATH, "intermediate_data/cpc/meetings-manifest.csv"))
DATES = sorted(list(meetings_df['date']))

In [3]:
for date in DATES:
    year = date[0:4]
    PATH = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}")
    print(f"{date}... ", end='')

    # Agenda df
    agenda_df = pd.read_pickle(os.path.join(PATH, 'agenda-items.pkl'))

    # Minutes full text
    try:
        with open(os.path.join(PATH, 'minutes-manual-override.txt'), 'r', encoding='utf-8') as f:
            minutes_text = f.read()
    except:
        with open(os.path.join(PATH, 'minutes.txt'), 'r', encoding='utf-8') as f:
            minutes_text = f.read()
    
    minutes_text_lines = minutes_text.split('\n')

    df = []
    j = 0
    for j in range(len(minutes_text_lines)):
        line = minutes_text_lines[j]
        relevant_items = []
        for idx, row in agenda_df.iterrows():
            found = False
            item_no = row['item_no']
            title = row['title']
            is_cc_heading = row['is_cc_heading']
            is_cc_part = row['is_cc_part']
            is_casenum = row['is_casenum']
            pattern = rf"^item no[\s.]*{item_no}$"
            pattern2 = rf"^item nos\..*\b{item_no}\b"
            pattern3 = rf"^item no[\s.]*0{item_no}$"
            if re.search(pattern, line.lower().strip()):
                start_line = j
                find_type = 'pattern1'
                found = True
            elif re.search(pattern2, line.lower().strip()):
                start_line = j
                find_type = 'pattern2'
                found = True
            elif re.search(pattern3, line.lower().strip()):
                start_line = j
                find_type = 'pattern3'
                found = True
            elif (is_cc_part) and (line.lower().strip()==item_no):
                start_line = j
                find_type = '<item_no> (cc part only)'
                found = True
            if found:
                relevant_items.append(item_no)
        if len(relevant_items)>0:
            df.append({
                'year': year,
                'date': date,
                'start_line': j,
                'relevant_items': relevant_items
            })
    df = pd.DataFrame.from_dict(df)
    
    # Get the end line for each item
    df['end_line'] = -1
    df['content'] = ''
    for idx, row in df.iterrows():
        start_line = row['start_line']
        if idx==len(df)-1:
            j = row['start_line']+1
            done = False
            while (not done) and (j<len(minutes_text_lines)):
                line = minutes_text_lines[j]
                if 'no further business' in line.lower():
                    done = True
                if done:
                    df.loc[idx, 'end_line'] = j-1
                    df.loc[idx, 'content'] = '\n'.join(minutes_text_lines[start_line:j])
                j+=1
            if j==len(minutes_text_lines):
                print(item_no, title)
                raise RuntimeError("couldn't find end line")
        else:
            end_line = df.loc[idx+1, 'start_line']
            df.loc[idx, 'end_line'] = end_line-1
            df.loc[idx, 'content'] = '\n'.join(minutes_text_lines[start_line:end_line])

    # Check if all the items were found and that their content contains self's case num
    df2 = []
    for idx, row in agenda_df.iterrows():
        item_no = row['item_no']
        title = row['title']
        is_cc_heading = row['is_cc_heading']
        is_cc_part = row['is_cc_part']
        is_casenum = row['is_casenum']

        found = False
        content = ''
        start_line = -1
        end_line = -1
        for jdx, row in df.iterrows():
            relevant_items = row['relevant_items']
            if item_no in relevant_items:
                found = True
                content = content + '\n' + row['content']
                start_line = row['start_line']
                end_line = row['end_line']
        if (not is_cc_heading) and (not found):
            print(item_no, title)
            raise RuntimeError('not all agenda items found')

        tokz = [tok for tok in content.split() if utils.is_casenum(tok)]
        if (is_casenum) and (len(tokz)>0) and (title not in tokz):
            print(item_no, title)
            print(content)
            raise RuntimeError('case num not found in minutes content')

        df2.append({
            'year': year,
            'date': date,
            'item_no': item_no,
            'title': title,
            'is_cc_heading': is_cc_heading,
            'is_cc_part': is_cc_part,
            'is_casenum': is_casenum,
            'start_line': start_line,
            'end_line': end_line,
            'content': content
        })
    df2 = pd.DataFrame.from_dict(df2)

    df.to_pickle(os.path.join(PATH, 'minutes.pkl'))
    df2.to_pickle(os.path.join(PATH, 'minutes-items.pkl'))


2018-05-10... 2018-05-23... 2018-06-14... 2018-07-12... 2018-07-26... 2018-08-09... 2018-08-23... 2018-09-13... 2018-09-27... 2018-10-11... 2018-10-25... 2018-11-08... 2018-11-29... 2018-12-13... 2018-12-20... 2019-01-10... 2019-01-24... 2019-02-14... 2019-02-28... 2019-03-14... 2019-03-28... 2019-04-11... 2019-05-09... 2019-05-23... 2019-06-13... 2019-06-27... 2019-07-11... 2019-07-25... 2019-08-08... 2019-08-22... 2019-09-12... 2019-09-26... 2019-10-10... 2019-10-24... 2019-11-14... 2019-11-21... 2019-12-12... 2019-12-19... 2020-01-09... 2020-01-23... 2020-02-13... 2020-03-12... 2020-04-23... 2020-05-14... 2020-05-28... 2020-06-04... 2020-06-11... 2020-06-25... 2020-07-09... 2020-07-23... 2020-08-13... 2020-08-27... 2020-09-10... 2020-09-17... 2020-09-24... 2020-10-08... 2020-10-15... 2020-10-22... 2020-11-05... 2020-11-19... 2020-12-03... 2020-12-10... 2020-12-17... 2021-01-14... 2021-01-28... 2021-02-11... 2021-02-18... 2021-02-25... 2021-03-11... 2021-03-25... 2021-04-08... 2021-0

In [4]:
#date = meetings_meta_df.sample(1).iloc[0]['date']
date = '2023-11-16'
year = date[0:4]
filepath = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}")
df = pd.read_pickle(os.path.join(filepath, 'minutes.pkl'))
df2 = pd.read_pickle(os.path.join(filepath, 'minutes-items.pkl'))

df

,year,date,start_line,relevant_items,end_line,content
0,2023,2023-11-16,33,[1],43,ITEM NO....
1,2023,2023-11-16,44,[2],50,ITEM NO....
2,2023,2023-11-16,51,[3],66,ITEM NO....
3,2023,2023-11-16,67,[4],78,ITEM NO....
4,2023,2023-11-16,79,[5a],132,ITEM NO. ...
5,2023,2023-11-16,133,[6],193,ITEM NO....
6,2023,2023-11-16,194,[6],262,ITEM NO....
7,2023,2023-11-16,263,[6],316,ITEM NO....
8,2023,2023-11-16,317,[7],385,ITEM NO....
9,2023,2023-11-16,386,[5b],468,ITEM NO. ...


In [5]:
print(df2.loc[6, 'content'])


                                      ITEM NO. 6                                    
                                                                                    
       CPC-2023-5273-CA                                        Council District: ALL
       CEQA: ENV-2020-6762-EIR; SCH No. 2021010130              Last Day to Act: N/A
            ENV-2020-6762-EIR-ADD1                                                  
       Plan Area: Citywide                                                          
                                                                                    
       PUBLIC HEARING – Completed on October 11, 2023                               
                                                                                    
       PROJECT SITE:  Citywide                                                      
                                                                                    
       IN ATTENDANCE:                                           